In [49]:
import numpy as np
import os

In [50]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [51]:
DATA_PATH = 'G:\signbot\DATA\MP_DATA'
actions = np.array(open('G:\signbot\DATA\classes.txt', 'r').read().split('\n'))
number_of_sequences = 100
every_sequence_length = 30
sequences, labels = [], []

In [52]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'1': 0, '2': 1, '3': 2, '': 3}

In [53]:
for action in actions:
    for sequence in range(number_of_sequences):
        window = []
        for frame_num in range(every_sequence_length):
            try:
                res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            # print(res)
            except:
                pass
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [54]:
# sequences[0]

In [55]:
X = np.array(sequences)

In [56]:
# X

In [57]:
# X.shape

In [58]:
y = to_categorical(labels).astype(int)

In [59]:
# y

In [60]:
# y.shape

In [61]:
X_train_and_val, X_test, y_train_and_val, y_test = train_test_split(X,y, train_size=0.9)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_and_val,y_train_and_val, test_size=0.2)

In [62]:
# X_train.shape

In [63]:
# X_valid.shape

In [64]:
# X_test.shape

In [65]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [66]:
batch_size = 128    
epochs = 1000

# log_dir = 'Logs', datetime.datetime.now().strftime("%Y.%m.%d-%H.%M") + '--batch__' + str(batch_size) + '--epochs__' + str(epochs)
# tb_callback = TensorBoard(log_dir=log_dir)

In [67]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))


In [68]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [69]:
model.fit(X_train, y_train, epochs=epochs, shuffle=True, batch_size=batch_size, validation_data= (X_valid, y_valid))

Epoch 1/1000
3/3 [==============================] - 6s 382ms/step - loss: 1.3842 - categorical_accuracy: 0.3854 - val_loss: 1.3805 - val_categorical_accuracy: 0.4722
Epoch 2/1000
3/3 [==============================] - 0s 91ms/step - loss: 1.3701 - categorical_accuracy: 0.3854 - val_loss: 1.3581 - val_categorical_accuracy: 0.4722
Epoch 3/1000
3/3 [==============================] - 0s 146ms/step - loss: 1.3360 - categorical_accuracy: 0.5139 - val_loss: 1.2909 - val_categorical_accuracy: 0.4722
Epoch 4/1000
3/3 [==============================] - 0s 126ms/step - loss: 1.2366 - categorical_accuracy: 0.5139 - val_loss: 1.0896 - val_categorical_accuracy: 0.4722
Epoch 5/1000
3/3 [==============================] - 0s 110ms/step - loss: 1.0108 - categorical_accuracy: 0.5139 - val_loss: 0.9634 - val_categorical_accuracy: 0.4722
Epoch 6/1000
3/3 [==============================] - 0s 137ms/step - loss: 0.8934 - categorical_accuracy: 0.5139 - val_loss: 0.8616 - val_categorical_accuracy: 0.4722
Epoch

In [70]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 30, 64)            48896     
_________________________________________________________________
lstm_7 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_8 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_7 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_8 (Dense)              (None, 4)                 132       
Total params: 203,492
Trainable params: 203,492
Non-trainable params: 0
________________________________________________

In [71]:
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=1)
print("test loss, test acc:", results)

print("Generate predictions for 30 samples")
predictions = model.predict(X_test)
print("predictions shape:", predictions.shape)

Evaluate on test data
40/40 [==============================] - 1s 6ms/step - loss: 0.2791 - categorical_accuracy: 0.8000
test loss, test acc: [0.2791467308998108, 0.800000011920929]
Generate predictions for 30 samples
predictions shape: (40, 4)


In [72]:
model.save('G:/signbot/MODEL/action1.h5')